0.Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


ERROR: tflite-model-maker 0.2.4 has requirement tensorflow>=2.4.0, but you'll have tensorflow 2.3.0 which is incompatible.
ERROR: tensorflowjs 3.0.0 has requirement tensorflow-hub<0.10,>=0.7.0, but you'll have tensorflow-hub 0.11.0 which is incompatible.


  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656450 sha256=15183d047fe3fd8b5e25e0b1718a27889504c442818c2ac5b5614f1fd1b9a4ef
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\99\f7\e3\d6f0f120ac047c1e5de2ae34930e7bf6e8de1c7a4d5fa68555
Successfully built gym


1.Test Random Environment with OpenAI Gym

In [2]:
import gym
import random

In [3]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
episodes = 10
for episode in range(1, episodes +1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        #env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score: {}'.format(episode, score))

Episode:1 Score: 15.0
Episode:2 Score: 13.0
Episode:3 Score: 49.0
Episode:4 Score: 56.0
Episode:5 Score: 30.0
Episode:6 Score: 10.0
Episode:7 Score: 30.0
Episode:8 Score: 24.0
Episode:9 Score: 19.0
Episode:10 Score: 14.0


2.Create a Deep Learning Model with keras

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [6]:
from tensorflow.keras.layers import Dense

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
model = build_model(states, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


3.Build Agent with Keras-RL

In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000: 1s  - E
53 episodes - episode_reward: 187.698 [99.000, 200.000] - loss: 6.440 - mae: 39.178 - mean_q: 78.811

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 198.740 [165.000, 200.000] - loss: 5.279 - mae: 38.746 - mean_q: 77.803

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: 1.0000
50 episodes - episode_reward: 198.420 [154.000, 200.000] - loss: 7.804 - mae: 41.050 - mean_q: 82.246

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: 1.0000
51 episodes - episode_reward: 197.882 [92.000, 200.000] - loss: 10.691 - mae: 41.499 - mean_q: 82.963

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 1.

In [19]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [20]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


4.Reloading Agent from Memory

In [21]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [22]:
del model
del dqn
del env

In [23]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [24]:
dqn.load_weights('dqn_weights.h5f')

In [25]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
